# Preprocessing for analyzing CMIP6 OMZ data in oxygen coordinates

## What is blocking this?

- the new cmip6_pp masking would make this a lot easier (with the labels)


## General notes
- CM4 is not working due to the wonky chunks. Ill try to squeeze it through anyways, because I do not want to invest much more work here. This will all work better in the cloud.

In [1]:
from cmip6_omz.upstream_stash import append_write_zarr

In [2]:
from xhistogram.xarray import histogram

In [3]:
import intake
import xarray as xr
import numpy as np
from cmip6_preprocessing.utils import cmip6_dataset_id
from cmip6_preprocessing.preprocessing import combined_preprocessing
from cmip6_preprocessing.postprocessing import (
    match_metrics,
    interpolate_grid_label,
    merge_variables,
    concat_experiments,
)
from cmip6_preprocessing.drift_removal import match_and_remove_trend
from fastprogress.fastprogress import progress_bar

from cmip6_omz.utils import cmip6_collection, o2_models
from cmip6_omz.upstream_stash import (
    pick_first_member,
    construct_static_dz
)
from cmip6_omz.units import convert_mol_m3_mymol_kg

from xarrayutils.file_handling import maybe_create_folder

### needs cleaning
from cmip6_omz.omz_tools import omz_thickness_efficient

In [4]:
import dask
from multiprocessing.pool import ThreadPool
dask.config.set(pool=ThreadPool(6))

In [5]:
# # set up dask
# from dask.distributed import LocalCluster, Client
# mem_total = 128
# workers = 2
# threads = 4 # 4*6 seemed to work quite well, but I would like this to perform a bit better
# cluster = LocalCluster(
#     memory_limit=f"{int(mem_total/workers)}GiB",
#     dashboard_address=9999,
#     threads_per_worker=threads,
#     n_workers = workers,
#                       )
# client = Client(cluster)
# client

In [6]:
ofolder = maybe_create_folder('/projects/GEOCLIM/LRGROUP/jbusecke/projects_data/cmip6_depth_histogram')

/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder /projects/GEOCLIM/LRGROUP/jbusecke/projects_data/cmip6_depth_histogram does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


In [7]:
o2_models()

['CanESM5-CanOE',
 'CanESM5',
 'CNRM-ESM2-1',
 'ACCESS-ESM1-5',
 'MPI-ESM-1-2-HAM',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'UKESM1-0-LL',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NorCPM1',
 'NorESM1-F',
 'NorESM2-LM',
 'NorESM2-MM',
 'GFDL-CM4',
 'GFDL-ESM4']

In [8]:
import dask.distributed
print(dask.distributed.__version__)

2021.08.1


In [9]:
# if this does not work on jupyter.rc, we can add some logic to
col = intake.open_esm_datastore(
    cmip6_collection(zarr=True)
)  # TODO: Check with nc files

z_kwargs = {"decode_times": True, "use_cftime": True, "consolidated": True}
n_kwargs = {"decode_times": True, "use_cftime": True, "chunks": {"time": 1}}

variable_ids = ["thetao", "so", "o2", "agessc"]  # "mlotst"
metric_variable_ids = ["thkcello", "areacello"]  # "mlotst"

# models = o2_models()
models = [
#      'CanESM5-CanOE',
#      'CanESM5',
#      'CNRM-ESM2-1',
#      'ACCESS-ESM1-5',
#      'MPI-ESM-1-2-HAM',
#      'IPSL-CM6A-LR',
#      'MIROC-ES2L',
#      'UKESM1-0-LL',
     'MPI-ESM1-2-HR',
#      'MPI-ESM1-2-LR',
#      'MRI-ESM2-0',
#      'NorCPM1',
#      'NorESM1-F',
#      'NorESM2-LM',
#      'NorESM2-MM',
#      'GFDL-CM4',
#      'GFDL-ESM4'
]
# models = [m for m in o2_models() if 'GFDL' not in m]
# models = ['ACCESS-ESM1-5']

cat = col.search(
    source_id=models,
    grid_label=["gr", "gn"],
    experiment_id=["historical", "ssp585"],
    table_id=["Omon"],
    variable_id=variable_ids,
)
ds_dict = cat.to_dataset_dict(
    aggregate=False,
    zarr_kwargs=z_kwargs,
    cdf_kwargs=n_kwargs,
    preprocess=combined_preprocessing,
)

# make a separate metric dict to catch all possible metrics!
cat_metrics = col.search(source_id=models, variable_id=metric_variable_ids)
ds_metric_dict = cat_metrics.to_dataset_dict(
    aggregate=False,
    zarr_kwargs=z_kwargs,
    cdf_kwargs=n_kwargs,
    preprocess=combined_preprocessing,
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'


In [10]:
ds

NameError: name 'ds' is not defined

In [ ]:
import pathlib
# new files (change in later and get rid of `load_trend_dict` (or refactor?) and `fix_trend_metadata`)
# Load all trend files
flist = list(pathlib.Path('../../data/external/cmip6_control_drifts/').absolute().glob('*.nc'))
flist = [f for f in flist if any([v in str(f) for v in variable_ids])]
trend_dict = {}
for f in progress_bar(flist):
    trend_dict[f.stem] = xr.open_mfdataset([f])

In [ ]:
# these ones are messed up...need a better way to deal with that in the previous step
# see https://github.com/jbusecke/cmip6_preprocessing/issues/175
incomplete_keys = ['CMIP.IPSL.IPSL-CM6A-LR.historical.r3i1p1f1.Omon.gn.none.area_o2']
trend_dict = {k:ds for k,ds in trend_dict.items() if k not in incomplete_keys}

ddict_tracers_detrended = match_and_remove_trend(
    ds_dict,
    trend_dict,
)

## Match metrics

In [ ]:
# this one causes problems because the time is not as long as the full data...
ddict_tracers_detrended_filtered = {
    k: ds.squeeze()
    for k, ds in ddict_tracers_detrended.items()
    if not ("CNRM-ESM2-1" in k and "r6i1p1f2" in k)
}

ddict_matched = match_metrics(
    ddict_tracers_detrended_filtered,
    ds_metric_dict,
    ["areacello", "thkcello"],
    print_statistics=True,
)

## Interpolate Grids and merge variables

- handle the Norwegian Models inside `interpolate_grid_label`

In [ ]:
print("interpolate grids\n")
ddict_matched_regrid = interpolate_grid_label(
    ddict_matched, merge_kwargs={"compat": "override"}
)  # This should be a default soon

In [ ]:
#patch the norwegian model in manually
ddict_patch = merge_variables(ddict_matched)
for name, ds in ddict_patch.items():
    if 'Nor' in name and 'gr' in name:
        patch_name = name.replace('.gr','')
        ddict_matched_regrid[patch_name] = ds

In [ ]:
np.sort(list(ddict_matched_regrid.keys()))

## Concatenate experiments and pick the first full one

In [ ]:
# somehow xarray cannot deal with comparing list/int attrs (Occurs in CM4)
# I should raise that, but lets fix it quickly here
def clean_attrs(ds):
    for a, attr in ds.attrs.items():
        if isinstance(attr, int):
            ds.attrs[a] = [attr]
    return ds

ddict_matched_regrid = {k:clean_attrs(ds) for k, ds in ddict_matched_regrid.items()}

ddict_ex_combined = concat_experiments(
    ddict_matched_regrid,
    concat_kwargs={
        'combine_attrs': 'drop_conflicts',
        'compat': 'override',
        'coords': 'minimal'
    }
)

In [ ]:
ddict_ex_combined.keys()

Still need to deal with the access stuff here...

In [ ]:
# only pick full runs (historical and ssp585)
ddict_ex_combined_full = {k:ds for k,ds in ddict_ex_combined.items() if len(ds.time)>3000}

## Check for problems and fix missing area/thickness manually

This should be wrapped and brought upstream

In [ ]:
from cmip6_preprocessing.grids import combine_staggered_grid
problems = {'missing_variables':[], 'missing_area':[], 'missing_thickness':[], 'reconstructed_area':[], 'reconstructed_thickness':[]}
ddict_filtered = {}
for name, ds in ddict_ex_combined_full.items():
    flag = False
    # Check that all necessary variables are given
    missing_variables = [va for va in ["thetao", "so", "o2"] if va not in ds.variables]
    if len(missing_variables)>0:
        flag = True
        problems['missing_variables'].append((name, missing_variables))
        
    # Check for area
    if not 'areacello' in ds.coords:
        if ds.attrs['grid_label'] == 'gr': # only reconstruct for regular grids
            grid, ds = combine_staggered_grid(ds, recalculate_metrics=True)
            # I am dropping dz_t here so it can be uniformly reconstructed
            ds = ds.drop('dz_t')
            ds = ds.assign_coords(areacello = (ds.dx_t * ds.dy_t).reset_coords(drop=True))
            problems['reconstructed_area'].append(name)
            assert 'areacello' in ds.coords
        else:
            flag = True
            problems['missing_area'].append(name)
    
    # Check for thickness (and rename) TODO: We should probably not rename and just refactor to use `thkcello`
    if "thkcello" in ds.coords:
        ds = ds.rename({'thkcello': 'dz_t'})
    else:
        # try to reconstruct the thickness from static info
        try:
#             lev_vertices = cf_xarray.bounds_to_vertices(ds.lev_bounds, 'bnds').load()
#             dz_t = lev_vertices.diff('lev_vertices')
#             ds = ds.assign_coords(dz_t=('lev', dz_t.data))
            ds = construct_static_dz(ds).rename({'thkcello': 'dz_t'})
            problems['reconstructed_thickness'].append(name)
        except Exception as e:
            print(f'{name} thickness reconstruction failed with {e}')
            print(ds)
            problems['missing_thickness'].append(name)
            flag=True
            
    if not flag:
        ddict_filtered[name] = ds

In [ ]:
ddict_final = pick_first_member(ddict_filtered)#
list(np.sort(list(ddict_final.keys())))

## Prep Basin mask
- Needs a separated Indian Ocean (Sam uses: 78E)
- Refactor with the new masking using cf-xarray

In [ ]:
# load ocean basin data
import regionmask
basins = regionmask.defined_regions.natural_earth.ocean_basins_50
from cmip6_preprocessing.regionmask import merged_mask

## Define bins

In [ ]:
o2_bins = np.hstack([-100, np.arange(5, 160, 5)])  # in mymol/kg
o2_bins_converted = bins_converted = (
    o2_bins / convert_mol_m3_mymol_kg(xr.DataArray([1])).data
)

# define mask bins
mask_bins = np.arange(-0.5, 13.0, 1)# for now manual, but maybe there is a clever way to do this?
mask_bins

lat_bins = np.arange(-60, 91, 20)
lat_bins

## This needs to go within the loop later


- I need to also make sure nans are appropriately masked between variables

In [ ]:
def check_exist(path):
#     return path.exists()
# hack for CM4
    return False

In [ ]:
for name, ds in ddict_final.items():
#     ds = ds.chunk({'time':1, 'x':720, 'y':576})
    ds = ds.reset_coords('dz_t')
    mask = merged_mask(basins, ds)
    mask.name = "basin_mask"

    # Create a dataset
    vol = ds.dz_t * ds.areacello

    count = histogram(
        ds.o2,
        ds.lat,
        mask,
        bins=[o2_bins_converted, lat_bins, mask_bins],
        dim=["x", "y"],
    )
    volume = histogram(
        ds.o2,
        ds.lat,
        mask,
        bins=[o2_bins_converted, lat_bins, mask_bins],
        weights=vol,
        dim=["x", "y"],
    )
    tracers = {}
    for tr in ds.data_vars:
        tracers[tr] = histogram(
            ds.o2,
            ds.lat,
            mask,
            bins=[o2_bins_converted, lat_bins, mask_bins],
            weights=ds[tr] * vol,
            dim=["x", "y"],
        )
    ds_hist = xr.Dataset(dict(count=count, volume=volume, **tracers))
    ds_hist.attrs = {k:v for k,v in ds.attrs.items() if k not in ['intake_esm_varname']}
    print(f"{ds_hist.nbytes/1e9} GB")
    path = ofolder.joinpath(f"{cmip6_dataset_id(ds_hist)}")
    if not check_exist(path):
        print(path)
#         append_write_zarr(ds_hist, path, 60)
        append_write_zarr(ds_hist, path, 1) # just for CM4

In [ ]:
stop here

## Test the output against something I know

This seems fine. The bin count is not exact, but that is probably due to some changes in the numerical precision in one of the methods.

Nice the mean tracer values also line up.

In [ ]:
# might need to rename a few here so that it works and the dataset should probably be synth 
# Then I can move this to the tests.

# define mask bins
mask_bins = np.arange(-0.5, 13.0, 1)# for now manual, but maybe there is a clever way to do this?
mask_bins

lat_bins = np.arange(-90, 91, 20)
lat_bins

lev_bins = np.arange(0, 7000, 500)
lev_bins
vol = (ds.dz_t*ds.areacello) 

count = histogram(ds.o2, ds.lat, mask, bins=[o2_bins_converted, lat_bins, mask_bins], dim=['x','y'])
volume = histogram(ds.o2, ds.lat, mask, bins=[o2_bins_converted, lat_bins, mask_bins], weights=vol, dim=['x','y'])
tracers = {}
for tr in ds.data_vars:
    tracers[tr] = histogram(ds.o2, ds.lat, mask, bins=[o2_bins_converted, lat_bins, mask_bins], weights=ds[tr]*vol, dim=['x','y'])

In [ ]:
from cmip6_omz.omz_tools import mask_basin

In [ ]:
# expected_threshold = 0.082
cutoff = o2_bins_converted[-1]
test_full_pacific = test
# expected_full_pacific = mask_basin(ds.o2.isel(time=0), drop=False)
expected_full_pacific = ds.o2
expected_full_pacific = xr.ones_like(expected_full_pacific).where(expected_full_pacific<=cutoff)

In [ ]:
test

In [ ]:
expected_full_pacific

In [ ]:
ds.o2.shape

In [ ]:
expected_full_pacific.shape

In [ ]:
test_full_pacific.sum().data/expected_full_pacific.sum().load().data

I suspect this is due to the numerial precision going wrong somewhere...

In [ ]:
test

In [ ]:
# example with masked basin
cutoff = o2_bins_converted[-3]
test_full_pacific = test.sel(basin_mask_bin=slice(1.5, 3.5))
expected_full_pacific = mask_basin(ds.o2, drop=False)
expected_full_pacific = xr.ones_like(expected_full_pacific).where(expected_full_pacific<=cutoff)

In [ ]:
test_full_pacific.sum().data/expected_full_pacific.sum().load().data

In [ ]:
# example with masked basin
cutoff = o2_bins_converted[-3]
test_full_pacific = test3.sel(basin_mask_bin=slice(1.5, 3.5)).sum()/test2.sel(basin_mask_bin=slice(1.5, 3.5)).sum()
expected_full_pacific = mask_basin(ds, drop=False)
expected_full_pacific = expected_full_pacific.where(expected_full_pacific<=cutoff)
expected_full_pacific = expected_full_pacific.o2.weighted((expected_full_pacific.areacello*expected_full_pacific.dz_t).fillna(0)).mean()

In [ ]:
expected_full_pacific.load()

In [ ]:
test_full_pacific

## synthetic example for xhistogram

I somehow cannot bin over 'lev'...ok for now. Problem is described in detail [here](https://github.com/xgcm/xhistogram/issues/16)

In [ ]:
# I cant get the count numbers to line up. What am I doing wrong here?
da = xr.DataArray(np.random.rand(400,76), name='test')
cutoff = 0.4
hist = histogram(da, bins=np.array([-1e3, cutoff]))
hist

In [ ]:
test = xr.ones_like(da).where(da<=cutoff).sum()
test

In [ ]:
da.plot()